# Imports

In [1]:
import mysql.connector as mysqlConnector
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pickle

# Connection à la base de données dataengineer via mysql connector 
On teste si la connection fonctionne et on affiche les bases de données contenues sur le serveur

In [2]:
conn = mysqlConnector.connect(host='Localhost',user='root',passwd='*****', database='dataengineer')
if conn:
    print("Connection Successful!")
else:
    print("Connection Failed!")
cur = conn.cursor()
cur.execute("SHOW DATABASES")
for row in cur:
    print(row)

Connection Successful!
('dataengineer',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('testwildcode',)


On affiche ici les tables contenues dans la bdd dataengineer

In [3]:
cur.execute("SHOW TABLES")
for row in cur:
    print(row)

('actor',)
('address',)
('category',)
('customer',)
('film',)
('film_actor',)
('film_category',)
('film_text',)
('inventory',)
('language',)
('payment',)
('rental',)
('staff',)
('store',)


Quelques lignes de la table address

In [4]:
cur.execute("SELECT * FROM address LIMIT 2")
for row in cur:
    print(row)

(1, '318 CHE DE ROUMAGOUA', 'LA CIOTAT', '13600')
(2, '19 RUE DES DAMES', 'SAINTE SAVINE', '10300')


# Passage de SQL à un dataframe pandas avec read_sql()
Cela va nous permettre de manipuler les données et d'y rajouter les colonnes latitude et longitude avec Nominatim

In [5]:
# Set SQL query as a comment
sql_query = ''' SELECT * FROM address '''

# Use pandas to pass sql query
df = pd.read_sql(sql_query, conn)

# Show the resulting DataFrame
df

,address_id,address,city,postal_code
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217
3,4,6 BD DES ETINES,LE COTEAU,42120
4,5,35 AV DU 159EME RIA,BRIANCON,5100
...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100
558,602,6 AV JEAN JAURES,FEURS,42110
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800


On verifie si il y a des données manquantes 

In [ ]:
df.info()

## Requêtage test de Nominatim avec une seule adresse 

In [8]:
locator = Nominatim(user_agent="myGPS")
location = locator.geocode("318 CHE DE ROUMAGOUA,13600,LA CIOTAT, France")

In [9]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 43.197642, Longitude = 5.6078904


## Création d'une copie du dataframe pour manipuler les données et modifier le dataframe

In [11]:
df_bis=df
df_bis

,address_id,address,city,postal_code
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217
3,4,6 BD DES ETINES,LE COTEAU,42120
4,5,35 AV DU 159EME RIA,BRIANCON,5100
...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100
558,602,6 AV JEAN JAURES,FEURS,42110
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800


### Création d'une nouvelle colonne 'full_address' issue de la concatenation de colonnes contenant toutes les informations des adresses

In [12]:
df_bis['full_address']=df_bis['address']+","+df_bis['postal_code']+","+df_bis['city']+",France"
df_bis

,address_id,address,city,postal_code,full_address
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,"318 CHE DE ROUMAGOUA,13600,LA CIOTAT,France"
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,"19 RUE DES DAMES,10300,SAINTE SAVINE,France"
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,"22 RUE AMIRAL GUEPRATTE,29217,LE CONQUET,France"
3,4,6 BD DES ETINES,LE COTEAU,42120,"6 BD DES ETINES,42120,LE COTEAU,France"
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA,5100,BRIANCON,France"
...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,"20 RUE COLI,93100,MONTREUIL,France"
558,602,6 AV JEAN JAURES,FEURS,42110,"6 AV JEAN JAURES,42110,FEURS,France"
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,"1 RUE HENRI RENAUDIN,8000,CHARLEVILLE-MEZIERES..."
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,"2 RUE DE LYON,93800,EPINAY SUR SEINE,France"


Lors du premier requêtage ces adresses ont causé une erreur, j'ai donc retiré l'abréviation pour remplacer par le mot entier

In [13]:
df_bis.loc[df_bis['address'].str.contains("GEN ")]

,address_id,address,city,postal_code,full_address
57,63,40 RUE GEN DE GAULLE,BAYEL,10310,"40 RUE GEN DE GAULLE,10310,BAYEL,France"
95,104,33 RUE DU GEN DE GAULLE,L'AIGLE,61300,"33 RUE DU GEN DE GAULLE,61300,L'AIGLE,France"


In [14]:
df_bis['full_address']=df_bis['full_address'].replace("GEN ","GENERAL ", regex=True)

In [15]:
df_bis.loc[df_bis['address'].str.contains("GEN ")]

,address_id,address,city,postal_code,full_address
57,63,40 RUE GEN DE GAULLE,BAYEL,10310,"40 RUE GENERAL DE GAULLE,10310,BAYEL,France"
95,104,33 RUE DU GEN DE GAULLE,L'AIGLE,61300,"33 RUE DU GENERAL DE GAULLE,61300,L'AIGLE,France"


Même scénario que précédemment ici le nom de la ville n'était pas complet

In [16]:
df_bis.loc[df_bis['city']=="SAINT-ANDRE"]

,address_id,address,city,postal_code,full_address
479,516,180 RUE SADI CARNOT,SAINT-ANDRE,59350,"180 RUE SADI CARNOT,59350,SAINT-ANDRE,France"


In [17]:
df_bis.loc[df['full_address']=="180 RUE SADI CARNOT,59350,SAINT-ANDRE,France", "full_address"] = "180 RUE SADI CARNOT,59350,SAINT-ANDRE-LEZ-LILLE,France"

In [18]:
df_bis.loc[df_bis['city']=="SAINT-ANDRE"]

,address_id,address,city,postal_code,full_address
479,516,180 RUE SADI CARNOT,SAINT-ANDRE,59350,"180 RUE SADI CARNOT,59350,SAINT-ANDRE-LEZ-LILL..."


## Requêtage final sur toutes les adresses. 
On utilise ici ratelimiter pour simuler un temps de latence entre les requêtes. Ici il est de 1 seconde ce qui est assez long mais la table ne contient que 500 lignes ce n'est pas embêtant. Le résultat de la requête sera contenu dans une nouvelle colonne 'location'.

In [19]:
# 1 - convenient function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df_bis['location'] = df_bis['full_address'].apply(geocode)
df_bis

,address_id,address,city,postal_code,full_address,location
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,"318 CHE DE ROUMAGOUA,13600,LA CIOTAT,France","(Chemin de Roumagoua, La Fieloula, La Ciotat, ..."
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,"19 RUE DES DAMES,10300,SAINTE SAVINE,France","(Rue des Dames, Sainte-Savine, Troyes, Aube, G..."
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,"22 RUE AMIRAL GUEPRATTE,29217,LE CONQUET,France","(Rue Amiral Guépratte, Le Croaë, Le Conquet, B..."
3,4,6 BD DES ETINES,LE COTEAU,42120,"6 BD DES ETINES,42120,LE COTEAU,France","(Boulevard des Etines, Le Coteau, Roanne, Loir..."
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA,5100,BRIANCON,France",None
...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,"20 RUE COLI,93100,MONTREUIL,France","(20, Rue Coli, Ruffins - Théophile-Sueur, Mont..."
558,602,6 AV JEAN JAURES,FEURS,42110,"6 AV JEAN JAURES,42110,FEURS,France","(6, Avenue Jean Jaurès, Feurs, Montbrison, Loi..."
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,"1 RUE HENRI RENAUDIN,8000,CHARLEVILLE-MEZIERES...","(1, Rue Henri Renaudin, La Bosse d'Etion, Char..."
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,"2 RUE DE LYON,93800,EPINAY SUR SEINE,France","(Rue de Lyon, Orgemont, Épinay-sur-Seine, Sain..."


In [20]:
df_bis['location'][0]

Location(Chemin de Roumagoua, La Fieloula, La Ciotat, Marseille, Bouches-du-Rhône, Provence-Alpes-Côte d'Azur, France métropolitaine, 13600, France, (43.197642, 5.6078904, 0.0))

Since our location object is a tuple (adress, point) to split it we have to transform it into a list and point is also a tuple (latitude, longitude, altitude)

In [21]:
df_bis['point'] = df_bis['location'].apply(lambda x: tuple(x.point) if x else None)
df_bis[['latitude','longitude','altitude']] = pd.DataFrame(df_bis['point'].tolist(), index=df_bis.index) 
df_bis

,address_id,address,city,postal_code,full_address,location,point,latitude,longitude,altitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,"318 CHE DE ROUMAGOUA,13600,LA CIOTAT,France","(Chemin de Roumagoua, La Fieloula, La Ciotat, ...","(43.197642, 5.6078904, 0.0)",43.197642,5.607890,0.0
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,"19 RUE DES DAMES,10300,SAINTE SAVINE,France","(Rue des Dames, Sainte-Savine, Troyes, Aube, G...","(48.2909051, 4.0457894, 0.0)",48.290905,4.045789,0.0
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,"22 RUE AMIRAL GUEPRATTE,29217,LE CONQUET,France","(Rue Amiral Guépratte, Le Croaë, Le Conquet, B...","(48.3613399, -4.7667997, 0.0)",48.361340,-4.766800,0.0
3,4,6 BD DES ETINES,LE COTEAU,42120,"6 BD DES ETINES,42120,LE COTEAU,France","(Boulevard des Etines, Le Coteau, Roanne, Loir...","(46.021659, 4.0921204, 0.0)",46.021659,4.092120,0.0
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA,5100,BRIANCON,France",None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,"20 RUE COLI,93100,MONTREUIL,France","(20, Rue Coli, Ruffins - Théophile-Sueur, Mont...","(48.868459, 2.468415, 0.0)",48.868459,2.468415,0.0
558,602,6 AV JEAN JAURES,FEURS,42110,"6 AV JEAN JAURES,42110,FEURS,France","(6, Avenue Jean Jaurès, Feurs, Montbrison, Loi...","(45.74356325, 4.226839162230233, 0.0)",45.743563,4.226839,0.0
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,"1 RUE HENRI RENAUDIN,8000,CHARLEVILLE-MEZIERES...","(1, Rue Henri Renaudin, La Bosse d'Etion, Char...","(49.7735463, 4.7088294, 0.0)",49.773546,4.708829,0.0
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,"2 RUE DE LYON,93800,EPINAY SUR SEINE,France","(Rue de Lyon, Orgemont, Épinay-sur-Seine, Sain...","(48.9538356, 2.2929192, 0.0)",48.953836,2.292919,0.0


On verifie s'il existe des données manquantes et il y en a. Nominatim n'a pas pu trouver des conrrespondances pour toutes les adresses.

In [22]:
df_bis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562 entries, 0 to 561
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address_id    562 non-null    int64  
 1   address       562 non-null    object 
 2   city          562 non-null    object 
 3   postal_code   562 non-null    object 
 4   full_address  562 non-null    object 
 5   location      461 non-null    object 
 6   point         461 non-null    object 
 7   latitude      461 non-null    float64
 8   longitude     461 non-null    float64
 9   altitude      461 non-null    float64
dtypes: float64(3), int64(1), object(6)
memory usage: 44.0+ KB


In [23]:
df_bis[df_bis['location'].isna()]

,address_id,address,city,postal_code,full_address,location,point,latitude,longitude,altitude
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA,5100,BRIANCON,France",None,None,NaN,NaN,NaN
8,9,35 RUE PORTE ST JEAN,ORLEANS,45000,"35 RUE PORTE ST JEAN,45000,ORLEANS,France",None,None,NaN,NaN,NaN
9,10,67 BD COMMANDANT CHARCOT,NEUILLY-SUR-SEINE,92200,"67 BD COMMANDANT CHARCOT,92200,NEUILLY-SUR-SEI...",None,None,NaN,NaN,NaN
13,14,117 RUE DES CHARMETTES,LYON 6EME,69006,"117 RUE DES CHARMETTES,69006,LYON 6EME,France",None,None,NaN,NaN,NaN
16,17,5 PL MARCEL PAGNOL,SAINT-NAZAIRE,44600,"5 PL MARCEL PAGNOL,44600,SAINT-NAZAIRE,France",None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
517,558,12 BD BOYER,MARSEILLE 3EME,13003,"12 BD BOYER,13003,MARSEILLE 3EME,France",None,None,NaN,NaN,NaN
527,568,131 AV FELIX FAURE,LYON 3EME,69003,"131 AV FELIX FAURE,69003,LYON 3EME,France",None,None,NaN,NaN,NaN
532,574,18 RUE DE COULON,SAINT-REMY,79410,"18 RUE DE COULON,79410,SAINT-REMY,France",None,None,NaN,NaN,NaN
533,575,2 IMP DE LA BREDE,SAINT-NAZAIRE,44600,"2 IMP DE LA BREDE,44600,SAINT-NAZAIRE,France",None,None,NaN,NaN,NaN


In [24]:
df_bis[(df_bis['location'].isna())&(df_bis['postal_code'].apply(lambda x: len(str(x))==4))]

,address_id,address,city,postal_code,full_address,location,point,latitude,longitude,altitude
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA,5100,BRIANCON,France",None,None,NaN,NaN,NaN
45,49,5 AV DOCTEUR FRANCOIS GOMMA,AX-LES-THERMES,9110,"5 AV DOCTEUR FRANCOIS GOMMA,9110,AX-LES-THERME...",None,None,NaN,NaN,NaN
51,56,95 AV DES FRERES ROUSTAN,VALLAURIS,6220,"95 AV DES FRERES ROUSTAN,6220,VALLAURIS,France",None,None,NaN,NaN,NaN
225,246,13 RUE DE NAUDIN,CONDE LES HERPY,8360,"13 RUE DE NAUDIN,8360,CONDE LES HERPY,France",None,None,NaN,NaN,NaN
376,407,12 RUE DU LT COL ANDRE BORDEREAUX,MOUZON,8210,"12 RUE DU LT COL ANDRE BORDEREAUX,8210,MOUZON,...",None,None,NaN,NaN,NaN


### Sauvegarde complète des données obtenues 

In [25]:
df_bis.to_pickle('address_geoloc.pkl')

In [29]:
df_loc=df_bis[['address_id','address','city','postal_code','latitude','longitude']]
df_loc

,address_id,address,city,postal_code,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,43.197642,5.607890
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,48.290905,4.045789
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,48.361340,-4.766800
3,4,6 BD DES ETINES,LE COTEAU,42120,46.021659,4.092120
4,5,35 AV DU 159EME RIA,BRIANCON,5100,NaN,NaN
...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,48.868459,2.468415
558,602,6 AV JEAN JAURES,FEURS,42110,45.743563,4.226839
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,49.773546,4.708829
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,48.953836,2.292919


In [47]:
df_loc.to_pickle('address_v1.pkl')